evaluated_model
```json
[
    {
        "question": "",
        "answer": ""
    }
]
```

In [1]:
from myutils import io
data = io.jsonload("data/eval.json")
evaluated_path = "evaluated_models/llama2-7bchat.json"
evaluated_model = io.jsonload(evaluated_path)

In [2]:
from src.double_loop import double_loop_match, double_loop_score
from tqdm import tqdm
assert len(data)==len(evaluated_model)
for data_unit, evaluated_model_unit in zip(data, evaluated_model):
    assert data_unit['question'] == evaluated_model_unit['question']
result = {
    "path": evaluated_path,
    "average_score": 0,
    "logs": []
}
sum_score = 0
for data_unit, evaluated_model_unit in tqdm(zip(data, evaluated_model)):
    question = data_unit['question']
    evaluated_answer = evaluated_model_unit['answer']
    scored_answers = [unit['answer'] for unit in data_unit['answers']]
    ys = [unit['score'] for unit in data_unit['answers']]
    xc, xs = double_loop_match(question, evaluated_answer, scored_answers)
    yc = double_loop_score(xc, xs, ys)
    sum_score += yc
    result['logs'].append({
        "xc": xc,
        "xs": xs,
        "yc": yc
    })
result['average_score'] = sum_score / len(data)
io.jsondump(result, evaluated_path+".log2")

122it [03:55,  1.93s/it]


In [7]:
import pandas as pd
from myutils import io
import os
def average(k: list):
    return sum(k)/len(k)
def getspan(a, b):
    return list(range(a, b))
def get_slice(a: list, indexes: list):
    result = []
    for i in indexes:
        result.append(a[i])
    return result
span1 = getspan(0, 30) + getspan(50, 60) + getspan(80, 90)
span2 = getspan(30, 50) + getspan(60, 70) + getspan(90, 108)
span3 = getspan(108, 122)
result = []
for file in os.listdir("evaluated_models"):
    if file.endswith(".log2"):
        filepath = os.path.join("evaluated_models", file)
        data = io.jsonload(filepath)
        result.append({
            'model': file.replace('.log2', ''),
            'language': average([unit['yc'] for unit in get_slice(data['logs'], span1)]),
            'memory': average([unit['yc'] for unit in get_slice(data['logs'], span2)]),
            'security': average([unit['yc'] for unit in get_slice(data['logs'], span3)]),
            'average': data['average_score']
        })
pd.DataFrame(result).sort_values('average', ascending=False)

,model,language,memory,security,average
4,gpt4.json,93.024160,89.301091,90.456978,88.988338
0,baichuan7bchat.json,87.522701,82.538491,90.138886,85.058719
3,gpt3.5.json,89.424512,82.980503,72.584380,83.620583
5,llama2-7bchat.json,94.168061,70.285417,96.405312,83.261408
6,qwen7bchat.json,85.398684,82.105599,82.410785,82.303568
1,chatglm-6b.json,82.685226,72.935230,75.752836,76.439790
2,chatglm2-6b.json,82.980736,65.797183,82.601787,75.068995
